# This notebook performs a goodlist/badlist substructure screening for a multi-molecule .sdf input file.  Also screens for rings of a minimum size, and maximum bond order.

## Enter screening parameters in the cell below.  Chemical input and output filenames should be in .sdf format, while goodlist and badlist files can be .txt files containing SMARTS patterns.

In [1]:
in_file = "C:\\Users\\umbci\\Desktop\\MAYGEN\\PC[0-3]O[1-2]H[3-5].sdf"
out_file = "C:\\Users\\umbci\\Desktop\\MAYGEN\\DEST_2.sdf"
goodlist = "C:\\Users\\umbci\\Desktop\\MAYGEN\\N-meGly+Ala_new2.txt" # substructures desired in final library
badlist = "C:\\Users\\umbci\\Desktop\\MAYGEN\\JoVE-MAYGEN_badlist_SMARTS.txt"
max_bond = 3 # only molecules with bond orders below this number will be kept
min_ring = 5 # only molecules with rings of this size or larger will be kept

### If you encounter a SyntaxError mentioning a unicode error, make sure there are no single backslashes (\\) in the file names.  Change any single backslashes to double backslashes (\\\\) or forward slashes (/).

## Module imports below

In [2]:
from rdkit import Chem

## Function definitions below

In [3]:
def minRingSize(mol, ring_min):
    """Determine presence of rings below a specified size.
    
    Determine if there are rings present (size 3 or greater) that are smaller than a user-defined minimum.

    Parameters
    ----------
    mol : RDKit Mol object
    
    ring_min : int
        Minimum ring size desired.  E.g. If ring_min = 6, then the function will look for rings of size 3, 4, and 5.

    Returns
    -------
    True if there are no rings below the specified minimum ring size; returns False otherwise.

    """
    ring_atoms = mol.GetRingInfo().AtomRings()
    sizes = set([len(a) for a in ring_atoms]) #lengths of all ring systems in molecule
    mins = set(list(range(3,ring_min)))
    return sizes.isdisjoint(mins)

In [4]:
def maxBonds(mol, bonds):
    """Test maximum bond order of input molecule.
    
    Tests if a molecule has a bond order equal to the input value.

    Parameters
    ----------
    mol : RDKit Mol object
    
    bonds : float
        Maximum bond order desired in a molecule.  Set as a float value because aromatic bonds in RDKit have a bond order of 2.5.

    Returns
    -------
    True if at least one bond in the molecule has the specified bond order.  Returns False otherwise.

    """
    bonds = float(bonds)
    max_bonds = False
    for b in mol.GetBonds():
        if b.GetBondTypeAsDouble() == bonds:
            max_bonds = True
            break
        else:
            continue
    return max_bonds

## Main program

In [5]:
%%time
# load in source data file
data = Chem.SDMolSupplier(in_file, removeHs=True)
data_set = set([Chem.MolToSmiles(m) for m in data]) # set of all input SMILES
bad_mols = set() # set of SMILES for molecules with unwanted substructures; output will lack these molecules
bad_substructs = []
for line in open(badlist, 'r'):
    bad_substructs.append(Chem.MolFromSmarts(line))
print("Molecules read: ", len(data_set))

Molecules read:  1928
CPU times: total: 281 ms
Wall time: 290 ms


In [6]:
%%time
# perform ring size screening
for mol in list(data_set):
    if not minRingSize(Chem.MolFromSmiles(mol), min_ring): #evaluates as True if molecule has a ring below the stated minimum size
        bad_mols.add(mol)
len(bad_mols)
# update data to search after each screening step, reducing subsequent screening time
data_set = (data_set - bad_mols)
print("Molecules left after ring screening: ", len(data_set))

Molecules left after ring screening:  672
CPU times: total: 156 ms
Wall time: 157 ms


In [7]:
%%time
# perform bond order screening
for mol in list(data_set):
    if maxBonds(Chem.MolFromSmiles(mol), max_bond):
        bad_mols.add(mol)
data_set = (data_set - bad_mols)
print("Molecules left after bond order screening: ", len(data_set))

Molecules left after bond order screening:  494
CPU times: total: 46.9 ms
Wall time: 48 ms


In [8]:
%%time
for mol in list(data_set):
    bad_chk = False
    for bad in bad_substructs:
        if not bad_chk:
            if Chem.MolFromSmiles(mol).HasSubstructMatch(bad):
                bad_chk = True
                break
            else:
                continue
        else:
            continue
    if bad_chk:
        bad_mols.add(mol)
data_set = (data_set-bad_mols)
print("Molecules left after removal of unwanted substructures: ", len(data_set))

Molecules left after removal of unwanted substructures:  254
CPU times: total: 2.17 s
Wall time: 2.18 s


## Use the following cell if there is a goodlist (file with SMARTS strings for substructures that must be included).  Otherwise, change the cell type from "Code" to "Markdown"

In [11]:
%%time
good_substructs = []
for line in open(goodlist, 'r'):
    good_substructs.append(Chem.MolFromSmarts(line))
for mol in list(data_set):
    #perform goodlist screening if backbone not represented by a phosphorous pseudoatom
    if ("P" in mol) or ("p" in mol):
        continue
    else:
        good_count = 0 # counts number of goodlist substructures in a molecule
        for good in good_substructs:
            if Chem.MolFromSmiles(mol).HasSubstructMatch(good):
                good_count += 1
        #if good_count < len(good_substructs): # retained structures must have all goodlist substructures, can be changed if desired
        if good_count == 0: # retained structures must have at least one goodlist substructure
            bad_mols.add(mol)
data_set = (data_set-bad_mols)
print("Molecules left after retention of desired substructures: ", len(data_set))

Molecules left after retention of desired substructures:  254
CPU times: total: 0 ns
Wall time: 1e+03 µs


### If you encounter a NameError in this cell for the 'goodlist' variable, you may have commented out the goodlist definition line in the first cell but ran this cell accidentally.  Change the cell type to "Markdown" by highlighting the cell and pressing "M", or in the menu at the top of the page select Cell --> Cell Type --> Markdown.

In [ ]:
# calculate and output filtered molecules
out_mols = list(data_set - bad_mols)
w = Chem.SDWriter(out_file)
for mol in out_mols:
    m2 = Chem.MolFromSmiles(mol)
    w.write(m2)
w.close()